In [69]:
%run APYME_bd.ipynb
path = '/Users/mb66483/Downloads/Crédito PyME/'


In [41]:
df_universo, df_sunat, df_sunart, df_rcc, df_ranking, df_digital, df_transac = read_bd(path)


db_ranking = clean_db_ranking(df_ranking)
db_ranking.sample(5)

df_sunart_1 = transf_sunart()
df_rcc_5 = transf_rcc()



/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [81]:
df_join = join_tables()
df_join = df_join[(df_join.veh_ct>0) & df_join.income_monthly_amount]

In [82]:
df = df_join
df_features = ['income_monthly_amount','birth_date','veh_ct','veh_year']
vo = 'vo_balance_amount'

X_train, X_test, y_train, y_test = partition(df,df_features,vo)

In [83]:
random_state=1
activation='relu'
y_predict, regr = model_nn(X_train, X_test, y_train, random_state, activation)

In [96]:
dfcom1 = y_test.reset_index()
dfcom2_1 = df[['personal_id_x','customer_id','sbs_customer_id','taxpayer_id','segmento','gender_type']].reset_index()


dfcom2 = pd.concat([dfcom1, pd.DataFrame(y_predict, columns=["valor_pred"])],axis=1)

dfcom2['error_est'] = dfcom2['vo_balance_amount']-dfcom2['valor_pred']
dfcom2['p_error_est'] = abs(dfcom2['error_est']/dfcom2['vo_balance_amount'])
    

In [97]:
dfcom2

,index,vo_balance_amount,valor_pred,error_est,p_error_est
0,1074,651891.69,800964.894291,-1.490732e+05,0.228678
1,1290,488916.34,254280.911167,2.346354e+05,0.479909
2,34104,2178809.60,280985.802973,1.897824e+06,0.871037
3,30525,-38889.12,323503.788191,-3.623929e+05,9.318619
4,7811,49099.30,285333.370029,-2.362341e+05,4.811353
...,...,...,...,...,...
1014,174,78320.35,930715.117000,-8.523948e+05,10.883439
1015,2485,903567.95,590001.052934,3.135669e+05,0.347032
1016,16939,141316.30,283078.495929,-1.417622e+05,1.003155
1017,559,403659.68,226059.980521,1.775997e+05,0.439974


In [89]:
dfcom1
df

,index,vo_balance_amount
0,1074,651891.69
1,1290,488916.34
2,34104,2178809.60
3,30525,-38889.12
4,7811,49099.30
...,...,...
1014,174,78320.35
1015,2485,903567.95
1016,16939,141316.30
1017,559,403659.68


In [ ]:
ersonal_id_x	customer_id	sbs_customer_id	taxpayer_id	segmento	gender_type

In [88]:
df

,personal_type_x,personal_id_x,customer_id,sbs_customer_id,taxpayer_id,segmento,gender_type,birth_date,marital_status_type,admission_date,...,comm_start_date,company_economic_activity_id,cutoff_date_y,personal_id,veh_ct,veh_year,situacion_credito,vo,vo_balance_amount,sm
0,L,90048822,98314621,6.200577e+09,81285667393,PNN,M,1961.0,S,2019-01-10,...,NaN,52348,2020-02-19,90048822.0,1.0,1966.0,VIGENTE,1.0,265687.77,1
2,L,39025845,44955658,8.268873e+09,11854064724,PNN,M,1962.0,C,1998-11-13,...,NaN,52190,2020-02-19,39025845.0,2.0,2009.0,VIGENTE,1.0,3339249.09,1
5,L,9350890,36781895,1.686121e+08,40040624596,PNN,M,1973.0,S,2018-12-12,...,NaN,52408,2020-02-19,9350890.0,3.0,1998.0,VENCIDO,0.0,-22937.95,2
6,L,9350890,36781895,1.686121e+08,40040624596,PNN,M,1973.0,S,2018-12-12,...,NaN,52408,2020-02-19,9350890.0,3.0,1998.0,VIGENTE,1.0,1176278.00,2
7,L,49400873,18301674,3.731068e+09,72277039581,PNN,F,1967.0,C,1998-02-23,...,2009-04-16,52310,2020-02-19,49400873.0,2.0,2014.0,VIGENTE,1.0,4049624.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38131,L,20277544,38949702,5.566360e+09,52829180541,PNN,F,1984.0,S,2003-06-26,...,NaN,52391,2020-02-19,20277544.0,1.0,2006.0,VIGENTE,1.0,121986.57,1
38132,L,24770931,48153143,7.842011e+09,80007916156,PNN,M,1984.0,C,2013-07-25,...,NaN,55205,2020-02-19,24770931.0,1.0,1992.0,VIGENTE,1.0,216333.99,1
38134,L,5116266,49815187,1.399267e+09,45333816586,PNN,F,1985.0,S,2007-05-01,...,2003-12-22,55205,2020-02-19,5116266.0,1.0,1996.0,VENCIDO,0.0,-268.33,2
38135,L,5116266,49815187,1.399267e+09,45333816586,PNN,F,1985.0,S,2007-05-01,...,2003-12-22,55205,2020-02-19,5116266.0,1.0,1996.0,VIGENTE,1.0,413446.98,2
